# Initialize the model
We follow the same model structure as in ISSM: [https://issm.jpl.nasa.gov](https://issm.jpl.nasa.gov)

The model `md` consists of 

We will go through the settings one by one

In [ ]:
using dJUICE

md = model()

# Generate Mesh

call `md=triangle(md,domainname,resolution)` to generate an unstructured triangular mesh based on the domain outline defined by `domainname`, with a characteristic length `resolution` for the mesh 

We start with a square domain of $[0, 10^6~\text{m}]\times[0,10^6~\text{m}]$ with $5\times10^3$~m mesh resolution 

**TODO**: 
1. change to triangle()
2. move exp, data, etc. to dJUICE folder
3. use IJulia.load(domainname) to show the exp file

In [ ]:
# Currently, triangle uses ISSM version, we need to fix the bug in triangle, and use the julia version
md = triangle2(md,issmdir()*"/test/Exp/Square.exp",50000.) 

# need to install GLMakie
plotmodel(md, "mesh")

# Set the geometry

**TODO**
1. Fix the print bug in notebook
2. add subplot

In [ ]:
hmin=300.
hmax=1000.
ymin=minimum(md.mesh.y)
ymax=maximum(md.mesh.y)
xmin=minimum(md.mesh.x)
xmax=maximum(md.mesh.x);

We set the thickness of the ice shelf to be 
$$H(x,y)=h_{\max} + \frac{h_{\min}-h_{\max}}{10^6}y+\frac{h_{\min}-h_{\max}}{10^7}x$$
where $h_\min=300$ and $h_\max=1000$

In [ ]:
md.geometry.thickness = hmax .+ (hmin-hmax)*(md.mesh.y .- ymin)./(ymax-ymin) .+ 0.1*(hmin-hmax)*(md.mesh.x .- xmin)./(xmax-xmin)
plotmodel(md, md.geometry.thickness)

Because the ice shelf is floating, we can determine the base of the ice by flotation criteria
$$base=-\frac{\rho_i}{\rho_w}*H$$

In [ ]:
md.geometry.base      = -md.materials.rho_ice/md.materials.rho_water*md.geometry.thickness
plotmodel(md, md.geometry.base)

The surface of the ice is then $s=base+H$

In [ ]:
md.geometry.surface   = md.geometry.base+md.geometry.thickness
plotmodel(md, md.geometry.surface)

To gurantee the ice shelf is completely floating, we set the sea bed elevation to be 10~m deeper than the base of the ice $b=base-10$

In [ ]:
md.geometry.bed       = md.geometry.base .-10
plotmodel(md, md.geometry.bed)

# Set intitial conditions

We set the whole domain to be ice covered.
The initial velocity is set to ISSM solution, which will accelerate the solution procedure for this example. However, in reality, the solution is unknonw, so one can set the initial condition to 0, or some other values with priori knowledge. 

**TODO**
1. move all data to dJUICE path
2. after going through the whole tutorial, let's try to change the initial condition to be 0, and solve again

In [ ]:
# set ice mask 
md = setmask(md,"all","")

#Initial velocity
x     = archread(issmdir()*"/test/Data/SquareShelfConstrained.arch","x")
y     = archread(issmdir()*"/test/Data/SquareShelfConstrained.arch","y")
vx    = archread(issmdir()*"/test/Data/SquareShelfConstrained.arch","vx")
vy    = archread(issmdir()*"/test/Data/SquareShelfConstrained.arch","vy")
index = Int.(archread(issmdir()*"/test/Data/SquareShelfConstrained.arch","index"))
md.initialization.vx=InterpFromMeshToMesh2d(index,x,y,vx,md.mesh.x,md.mesh.y,0.0)
md.initialization.vy=InterpFromMeshToMesh2d(index,x,y,vy,md.mesh.x,md.mesh.y,0.0)
vel = (md.initialization.vx.^2+ md.initialization.vy.^2).^0.5
# plot the velocity magenitude
plotmodel(md, vel)

# Set physical parameters

The ice in large scale is a non-Neutownian, viscos fluid, governed by Glen's flow law
$$\sigma = \mu\tau$$
where $\mu$ is the visocity defined as 
$$\mu=B\tau_e^\frac{1-n}{2n}$$
where the pre-factor $B$ and exponent $n$ need to be defined in dJUICE

**TODO**
1. add SSA equation
1. add Cuffey
2. add why there is sliding

In [ ]:
md.materials.rheology_B=1.815730284801701e+08*ones(md.mesh.numberofvertices)
md.materials.rheology_n=3*ones(md.mesh.numberofelements);

The sliding velocity depends on the basal shear stress by a friciton law, here we use Budd friction law
$$\tau_b=C^2N^\frac{q}{p}|u_b|^{\frac{1}{q}-1}u_b$$
where $N$ is the effective pressure at the base of the ice, in this example, we use $p=1$ and $q=1$

**TODO** 
1. add formula of N

In [ ]:
md.friction.coefficient=20*ones(md.mesh.numberofvertices)
md.friction.p=ones(md.mesh.numberofvertices)
md.friction.q=ones(md.mesh.numberofvertices);

# Boundary conditions

The Dirichlet type boundary conditions are set in the same way as in ISSM

In [ ]:
md.stressbalance.spcvx = NaN*ones(md.mesh.numberofvertices)
md.stressbalance.spcvy = NaN*ones(md.mesh.numberofvertices)
pos = findall(md.mesh.vertexonboundary)
md.stressbalance.spcvx[pos] .= 0.0
md.stressbalance.spcvy[pos] .= 0.0;

# Numerical tolerance

- `restol` is the mechanical equilibrium residual convergence criterion
- `reltol` is the velocity relative convergence criterion
- `abstol` is the velocity absolute convergence criterion

If the tolerance is set to `NaN`, that means it is not applied.

In [ ]:
md.stressbalance.restol=0.05
md.stressbalance.reltol=0.05
md.stressbalance.abstol=NaN;

# Solve

Now let's solve the nonlinar PDEs by iterative method

In [ ]:
md=solve(md,:Stressbalance)

# plot solutions

**TODO**
1. change Vel, Vx, Vy in results to m/yr by default

In [ ]:
plotmodel(md, md.results["StressbalanceSolution"]["Vel"]*md.constants.yts)

# Rerun the example with 0 initial guess